In [ ]:
!pip uninstall -y numpy scipy transformers accelerate sentence-transformers #bertopic

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1


In [ ]:
!pip install numpy==1.26.4 umap-learn openai hdbscan accelerate scipy gensim nltk wordcloud pyLDAvis ipykernel langchain transformers==4.40.2 accelerate==0.27.2 sentence-transformers==2.5.1 bertopic==0.16.4 faiss-cpu  langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import umap
from umap import UMAP
from hdbscan import HDBSCAN
from collections import defaultdict
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel
from sentence_transformers import SentenceTransformer, models


from bertopic import BERTopic
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from bertopic.representation import KeyBERTInspired
from langchain.text_splitter import RecursiveCharacterTextSplitter


import string
import spacy
import umap
import re
import ast
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import joblib
import pickle
import torch
import os

#pd.set_option('display.max_colwidth', None)
#pd.reset_option('all')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


%cd /content/drive/My Drive/EP-BoE/

Mounted at /content/drive/
/content/drive/My Drive/EP-BoE


In [ ]:
df_qna = pd.read_csv("qna_df.csv")


In [ ]:
# Step 1: Identify where the 'content_type' changes
df_qna['group'] = (df_qna['content_type'] != df_qna['content_type'].shift()).cumsum()

# Step 2: Group by consecutive same 'content_type' blocks
result = df_qna.groupby(['group', 'content_type'], as_index=False).agg({
    'content': lambda x: ' '.join(x),
    'year' : lambda x: max(x),
    'quarter' : lambda x: max(x)
    })


# Optional: Drop the helper column if needed
 #= result.drop(columns='group')
df_qna.drop(columns='group')
result

,group,content_type,content,year,quarter
0,1,question,"so, trading question, i mean, markets busines...",2024,3
1,2,answer,"i appreciate the question, glenn, and i mean,...",2024,3
2,3,question,i appreciate that. this one will be a short f...,2024,3
3,4,answer,"glenn, its denis. i guess what i would sugges...",2024,3
4,5,question,i just had a follow-up ﬁrst on trading and ma...,2024,3
...,...,...,...,...,...
633,634,answer,sure. thank you for that. in terms of a quart...,2022,3
634,635,question,very clear . and then a separate follow-up ju...,2022,3
635,636,answer,"so i appreciate the question, jeremy . i mean...",2022,3
636,637,question,i guess the word for the quarter is resilienc...,2022,3


In [ ]:
# Extract odd rows (0, 2, 4...) — content and label
question = result['content'].iloc[::2].reset_index(drop=True)
year = result['year'].iloc[::2].reset_index(drop=True)
quarter = result['quarter'].iloc[::2].reset_index(drop=True)

# Extract even rows (1, 3, 5...) — content only
answer = result['content'].iloc[1::2].reset_index(drop=True)

# Create new DataFrame
df_qna_GS = pd.DataFrame({
    'Year': year,
    'Quarter': quarter,
    'Question': question,
    'Answer': answer

})

df_qna_GS

,Year,Quarter,Question,Answer
0,2024,3,"so, trading question, i mean, markets busines...","i appreciate the question, glenn, and i mean,..."
1,2024,3,i appreciate that. this one will be a short f...,"glenn, its denis. i guess what i would sugges..."
2,2024,3,i just had a follow-up ﬁrst on trading and ma...,"yes. sure, ebrahim, and i appreciate the ques..."
3,2024,3,understood. and just a follow-up on -- back t...,"well, i appreciate the question. and we've be..."
4,2024,3,can you hear me? i just have a question on th...,"when you have leading market shares, our glob..."
...,...,...,...,...
314,2022,3,"david, you talked about the need for deposits...","so thank you, gerard. it's denis. so i think ..."
315,2022,3,very good. and then as a follow-up in your ou...,"i don't think it's simple for sure, gerard. i..."
316,2022,3,"you've covered a lot of ground, so i've just ...",sure. thank you for that. in terms of a quart...
317,2022,3,very clear . and then a separate follow-up ju...,"so i appreciate the question, jeremy . i mean..."


In [ ]:
df_qna_GS["SL"]= range(df_qna_GS.shape[0])
df_qna_GS["ID"]= df_qna_GS["Year"].astype(str)+"_"+ df_qna_GS["Quarter"].astype(str)+"_"+ df_qna_GS["SL"].astype(str)
df_qna_GS

,Year,Quarter,Question,Answer,SL,ID
0,2024,3,"so, trading question, i mean, markets busines...","i appreciate the question, glenn, and i mean,...",0,2024_3_0
1,2024,3,i appreciate that. this one will be a short f...,"glenn, its denis. i guess what i would sugges...",1,2024_3_1
2,2024,3,i just had a follow-up ﬁrst on trading and ma...,"yes. sure, ebrahim, and i appreciate the ques...",2,2024_3_2
3,2024,3,understood. and just a follow-up on -- back t...,"well, i appreciate the question. and we've be...",3,2024_3_3
4,2024,3,can you hear me? i just have a question on th...,"when you have leading market shares, our glob...",4,2024_3_4
...,...,...,...,...,...,...
314,2022,3,"david, you talked about the need for deposits...","so thank you, gerard. it's denis. so i think ...",314,2022_3_314
315,2022,3,very good. and then as a follow-up in your ou...,"i don't think it's simple for sure, gerard. i...",315,2022_3_315
316,2022,3,"you've covered a lot of ground, so i've just ...",sure. thank you for that. in terms of a quart...,316,2022_3_316
317,2022,3,very clear . and then a separate follow-up ju...,"so i appreciate the question, jeremy . i mean...",317,2022_3_317


In [ ]:
########################################################################################################################################################################################################
########################################################################################################################################################################################################
########################################################################################################################################################################################################
########################################################################################################################################################################################################

# Mistral Model

In [ ]:
!pip install -q huggingface_hub

In [ ]:
from huggingface_hub import login
login("hf_udAWwftgeKkhgaqbTYMOhyqMSJgsGqFaVuq")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
# Load Mistral 7B Instruct model on CPU
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, device_map={"": "cpu"})

# Create a text-generation pipeline
mistral_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.01,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.15
)

# Wrap with LangChain LLM interface
llm = HuggingFacePipeline(pipeline=mistral_pipe)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/tmp/ipython-input-10-2988818963.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=mistral_pipe)


In [ ]:
# Text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Question generation prompt
question_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Extract only the exact question the speaker indend asking in the paragraph below.\n"
        "Return only the question. Do not include any explanation, instructions, or formatting.\n\n"
        "Paragraph:\n{text}\n\nQuestion:"
    )
)

question_chain = LLMChain(llm=llm, prompt=question_prompt)


/tmp/ipython-input-11-1804134573.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_chain = LLMChain(llm=llm, prompt=question_prompt)


In [ ]:
# New

def generate_questions(paragraph):
    result = question_chain.run(text=paragraph).strip()
    return result.split("Question:")[-1].strip()


df_qna_GS["Extracted_Question"]= df_qna_GS.head(10)["Question"].apply(generate_questions)

/tmp/ipython-input-14-4094761688.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = question_chain.run(text=paragraph).strip()
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
df_qna_GS.head(12)

,Year,Quarter,Question,Answer,SL,ID,Extracted_Question
0,2024,3,"so, trading question, i mean, markets busines...","i appreciate the question, glenn, and i mean,...",0,2024_3_0,What are your comments on the regulatory perce...
1,2024,3,i appreciate that. this one will be a short f...,"glenn, its denis. i guess what i would sugges...",1,2024_3_1,Should I take a historical ROE-ish on that cap...
2,2024,3,i just had a follow-up ﬁrst on trading and ma...,"yes. sure, ebrahim, and i appreciate the ques...",2,2024_3_2,What is the competitive threat posed by non-ba...
3,2024,3,understood. and just a follow-up on -- back t...,"well, i appreciate the question. and we've be...",3,2024_3_3,What are the building blocks for Goldman Sachs...
4,2024,3,can you hear me? i just have a question on th...,"when you have leading market shares, our glob...",4,2024_3_4,How are you thinking about the broader competi...
5,2024,3,the ﬁrst question relates to why do you stop ...,"yes. thanks, mike. i mean i know there's a lo...",5,2024_3_5,Why do you stop platform solution sets of busi...
6,2024,3,"well, back to the core business then, you sai...","i mean, one, i think there are long-term secu...",6,2024_3_6,To what degree are 10-year averages less relev...
7,2024,3,"can you hear me okay? okay , super . yes, i t...","sure, betsy . very good question. so generall...",7,2024_3_7,What is the relationship between the reduced p...
8,2024,3,"okay , super . right, because it's collateral...","sure, betsy . so, in terms of timing, it's si...",8,2024_3_8,When do you think GM Cut will close and are th...
9,2024,3,i wanted to follow up a little bit on the inv...,"so, thanks for the question, brennan. i mean ...",9,2024_3_9,What is your take on the recent success of spo...


In [ ]:
df_qna_GS.to_csv("df_qna_GS_exQ.csv")

In [ ]:
df_qna_GS_exQ=pd.read_csv("df_qna_GS_exQ.csv")

In [ ]:
def generate_questions(paragraph, max_questions=2):
    result = question_chain.run(text=paragraph).strip()

    # Split by line if multiple questions are returned
    questions = [q.strip('- ').strip() for q in result.split('\n') if q.strip()]

    # Optional: filter for actual question-like lines
    questions = [q for q in questions if '?' in q]

    return questions[:max_questions]

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-19-2964994661.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Run on your dataframe (assuming df_qna_GS is already defined)
results = []

for idx, row in df_qna_GS.head(1).iterrows():
    paragraph = row["Question"]
    doc_text = row["Answer"]

    questions = generate_questions(paragraph)
    questions = questions[1]
    chunks = splitter.split_text(doc_text)
    vectordb = FAISS.from_texts(chunks, embedding=embedding_model)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    )

    for q in questions:
        answer = qa_chain.run(q)
        results.append({
            "paragraph": paragraph,
            "question": q,
            "answer": answer
        })




results_df = pd.DataFrame(results)
results_df

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# New

# Run on your dataframe (assuming df_qna_GS is already defined)
results = []

def extract_answer(Question, Answer):
  chunks = splitter.split_text(Answer)
  vectordb = FAISS.from_texts(chunks, embedding=embedding_model)

  qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    )

  answer = qa_chain.run(Question)

  return answer

In [ ]:
df_qna_GS_exQ["Extracted_Answer"] = df_qna_GS_exQ.head(2).apply(lambda x:  extract_answer(x["Extracted_Question"], x["Answer"]),axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
